### IMPORTING LIBRARIES

In [172]:
import pandas as pd
import random
import numpy as np
from collections import Counter
import re
from matplotlib import pyplot as plt
import json
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import sys
import spacy
import gensim
from gensim.models import Word2Vec,FastText
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, ne_chunk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [81]:
# Tokenization (splitting text into words/sentences)
nltk.data.path.append("C:/Users/DELL8/AppData/Roaming/nltk_data")
nltk.download('punkt', download_dir="C:/Users/DELL8/AppData/Roaming/nltk_data")
nltk.download('stopwords')  # Removing common words (e.g., "the", "is", "and")
nltk.download('averaged_perceptron_tagger')  # POS tagging (identifying nouns, verbs, adjectives)
nltk.download('wordnet')  # WordNet database for synonyms and lemmatization
nltk.download('maxent_ne_chunker')  # Named Entity Recognition (NER) to identify names/locations
nltk.download('words')  # Dictionary of valid English words (used for spell checking, NER)
nltk.download('omw-1.4')  # Open Multilingual WordNet (support for multiple languages)


[nltk_data] Downloading package punkt to
[nltk_data]     C:/Users/DELL8/AppData/Roaming/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-

True

In [82]:
import pandas as pd
data= pd.read_csv("D:\\DELL8\\Documents\\data_tweets.csv")
data.head()


Unnamed: 0         username  \
0           0      bollybubble   
1           1    sushantssmile   
2           2   Namita67793491   
3           3  being_grapher04   
4           4    sushantssmile   

                                         description             location  \
0  Bollywood Bubble - Your One Stop Destination F...  Mumbai, Maharashtra   
1  ᴡᴀɴᴛɪɴɢ ᴛᴏ ʟɪᴠᴇ ᴀs ʏᴏᴜʀ sᴍɪʟᴇ\nᴊᴜsᴛ ᴏɴᴇ ʟᴀsᴛ ᴛ...                    ∞   
2                                    Justice for SSR                  NaN   
3  Photographer.\nSushant Singh Rajput's Fan\nSSR...                 भारत   
4  ᴡᴀɴᴛɪɴɢ ᴛᴏ ʟɪᴠᴇ ᴀs ʏᴏᴜʀ sᴍɪʟᴇ\nᴊᴜsᴛ ᴏɴᴇ ʟᴀsᴛ ᴛ...                    ∞   

   following  followers  totaltweets  retweetcount  \
0        440     132406        51202             1   
1        369        412        11591            17   
2         66         89        20550             5   
3         82         27           20             0   
4        369        412        11591             1   

                                                text  \
0  Sahil Shah identified as prime suspect in the ...   
1  Somewhere' between neurons &amp; narratives......   
2  Ppl trending National Embarrassment CBI - U R ...   
3  We all are missing your need. \n#SushantSinghR...   
4  Hi All,\n\nPlease sign this petition.\n\n#Just...   

                                            hashtags  
0  ['SushantSinghRajput', 'SahilShah', 'Bollywood...  
1  ['SushantSinghRajput', 'sushanting', 'thebesth...  
2                                                 []  
3  ['SushantSinghRajput', 'JusticeForSushantSingh...  
4  ['JusticeForSushantSinghRajput', 'SushantSingh...

#### **Dataset Description**
This dataset contains **3,000** records of Twitter data related to a specific topic. It includes user information, tweet content, and engagement metrics.

#### **Column Descriptions**
1. **`Unnamed: 0`** (int): Index column (can be ignored or removed).
2. **`username`** (string): The Twitter handle of the user who posted the tweet.
3. **`description`** (string, nullable): The profile description of the user.
4. **`location`** (string, nullable): The location provided by the user in their Twitter profile.
5. **`following`** (int): The number of accounts this user follows.
6. **`followers`** (int): The number of followers this user has.
7. **`totaltweets`** (int): The total number of tweets posted by this user.
8. **`retweetcount`** (int): The number of times this tweet has been retweeted.
9. **`text`** (string): The content of the tweet.
10. **`hashtags`** (list, string): The hashtags used in the tweet, stored as a string representation of a list.

#### **Missing Values**
- `description` has **476 missing values**.
- `location` has **1,581 missing values**.

#### **Possible Data Preprocessing**
- Drop or ignore the `Unnamed: 0` column.
- Handle missing values in `description` and `location` (e.g., fill with "Unknown" or remove).
- Convert `hashtags` from a string to an actual Python list.


#### Emoji Processing Code

In [83]:
def emoji(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :') , :O
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|:O)', ' positiveemoji ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' positiveemoji ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' positiveemoji ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-; , @-)
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;|@-\))', ' positiveemoji ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:, :-/ , :-|
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:|:-/|:-\|)', ' negetiveemoji ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' negetiveemoji ', tweet)
    return tweet


##### **Observation**
The dataset consists of tweets with various attributes such as username, location, follower count, and hashtags. Among these, the `text` column contains the actual tweet content, which may include emojis representing different sentiments. Since raw emojis may not contribute directly to machine learning models, replacing them with sentiment-based labels (`positiveemoji` and `negetiveemoji`) ensures better sentiment classification. The dataset also contains missing values in the `description` and `location` columns, which may require handling before further analysis.

##### **Conclusion**
The `emoji()` function standardizes emojis in tweets, making them easier to analyze in sentiment classification tasks. It replaces smiley faces, laughing symbols, hearts, and winks with `"positiveemoji"`, while frowning faces, crying symbols, and neutral expressions are mapped to `"negetiveemoji"`. However, there is a small spelling inconsistency (`negetiveemoji` should be `negativeemoji`). Overall, this preprocessing step is crucial in ensuring that emojis contribute effectively to textual sentiment analysis by making them interpretable for machine learning models.

In [84]:
with open("D:\DELL8\Documents\contractions.json", 'r') as f:
    contractions_dict = json.load(f)
contractions = contractions_dict


#### Tweet Processing Code

In [85]:
import re

def process_tweet(tweet):
    tweet = tweet.lower()                                             # Lowercases the string
    tweet = re.sub('@[^\s]+', '', tweet)                              # Removes usernames
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', tweet)   # Remove URLs
    tweet = re.sub(r"\d+", " ", str(tweet))                           # Removes all digits
    tweet = re.sub('&quot;'," ", tweet)                               # Remove (&quot;) 
    tweet = emoji(tweet)                                              # Replaces Emojis
    tweet = re.sub(r"\b[a-zA-Z]\b", "", str(tweet))                   # Removes all single characters
    for word in tweet.split():
        if word.lower() in contractions:
            tweet = tweet.replace(word, contractions[word.lower()])   # Replaces contractions
    tweet = re.sub(r"[^\w\s]", " ", str(tweet))                       # Removes all punctuations
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)                         # Convert more than 2 letter repetitions to 2 letter
    tweet = re.sub(r"\s+", " ", str(tweet))                           # Replaces double spaces with single space    
    return tweet


##### **Observation**
The dataset contains tweets with various textual elements, including usernames, URLs, emojis, special characters, and contractions. These elements can introduce noise into text analysis and affect sentiment classification. The `text` column, which holds the actual tweets, requires cleaning to enhance model performance. Additionally, since contractions (e.g., "can't" → "cannot") are present in tweets, handling them ensures consistency in textual data.

##### **Conclusion**
The `process_tweet()` function performs essential text preprocessing to clean and standardize tweets for sentiment analysis. It removes usernames, URLs, digits, special characters, and single-letter words while also handling emojis through the `emoji()` function. Contractions are expanded using a predefined dictionary to maintain consistency. Furthermore, it reduces character repetitions and ensures proper spacing. These preprocessing steps are crucial for improving the quality of textual data, making it more suitable for sentiment classification and other natural language processing (NLP) tasks.

In [196]:
data['processed_tweet'] = np.vectorize(process_tweet)(data['text'])
data.head()


text  \
0  Sahil Shah identified as prime suspect in the ...   
1  Somewhere' between neurons &amp; narratives......   
2  Ppl trending National Embarrassment CBI - U R ...   
3  We all are missing your need. \n#SushantSinghR...   
4  Hi All,\n\nPlease sign this petition.\n\n#Just...   

                                     processed_tweet  \
0  sahil shah identified as prime suspect in the ...   
1  somewhere between neurons amp narratives susha...   
2  ppl trending national embarrassment cbi the em...   
3  we all are missing your need sushantsinghrajpu...   
4  hi all please sign this petition justiceforsus...   

                                        cleaned_text    neg    neu    pos  \
0  sahil shah identified as prime suspect in the ...  0.145  0.855  0.000   
1  somewhere between neurons amp narratives susha...  0.000  1.000  0.000   
2  ppl trending national embarrassment cbi the em...  0.179  0.741  0.080   
3  we all are missing your need sushantsinghrajpu...  0.128  0.872  0.000   
4  hi all please sign this petition justiceforsus...  0.139  0.723  0.139   

   compound sentiment  
0   -0.2960  negative  
1    0.0000   neutral  
2   -0.6369  negative  
3   -0.2960  negative  
4    0.0000   neutral

### Stopword

In [87]:
stop_words = set(stopwords.words("english"))
original_texts = []
stopword_removed_texts = []

for text in data["text"].dropna(): 
    clean_text = re.sub(r"<.*?>", " ", text)  
    clean_text = re.sub(r"https?://\S+", " ", clean_text)  
    clean_text = re.sub(r"[^a-zA-Z\s]", " ", clean_text)  

    words = clean_text.lower().split()

    if not words:
        continue
    filtered_words = [word for word in words if word not in stop_words]

    original_texts.append(clean_text)
    stopword_removed_texts.append(" ".join(filtered_words))

df_stopwords = pd.DataFrame({"Original Text": original_texts, "Without Stopwords": stopword_removed_texts})
df_stopwords.head()

Original Text  \
0  Sahil Shah identified as prime suspect in the ...   
1  Somewhere  between neurons  amp  narratives   ...   
2  Ppl trending National Embarrassment CBI   U R ...   
3  We all are missing your need  \n SushantSinghR...   
4  Hi All \n\nPlease sign this petition \n\n Just...   

                                   Without Stopwords  
0  sahil shah identified prime suspect drug probe...  
1  somewhere neurons amp narratives sushantsinghr...  
2  ppl trending national embarrassment cbi u r em...  
3  missing need sushantsinghrajput justiceforsush...  
4  hi please sign petition justiceforsushantsingh...

##### **Observation**
The dataset contains tweets that often include unnecessary words, such as common stopwords ("the", "is", "in", "and"), HTML tags, URLs, and non-alphabetic characters. The presence of such elements can introduce noise, making it difficult for machine learning models to extract meaningful insights. Since stopwords do not contribute significantly to sentiment analysis, their removal helps improve text clarity and model efficiency.

##### **Conclusion**
The given code effectively processes tweets by eliminating HTML tags, URLs, and non-alphabetic characters, followed by the removal of stopwords. This ensures that the cleaned tweets retain only meaningful words, reducing dimensionality and enhancing the quality of textual features. The resulting dataframe (`df_stopwords`) provides both the original and stopword-free versions of each tweet, making it easier to compare and analyze the impact of stopword removal in NLP-based sentiment analysis.

### lemmatizer

In [88]:
lemmatizer = WordNetLemmatizer()
original_texts = []
lemmatized_texts = []

for text in data["text"].dropna(): 
    clean_text = re.sub(r"<.*?>", " ", text)  
    clean_text = re.sub(r"https?://\S+", " ", clean_text)  
    clean_text = re.sub(r"[^a-zA-Z\s]", " ", clean_text)  

    words = clean_text.lower().split()

    if not words:
        continue

    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    original_texts.append(clean_text)
    lemmatized_texts.append(" ".join(lemmatized_words))




In [89]:
df_lemmatized = pd.DataFrame({"Original Text": original_texts, "Lemmatized Text": lemmatized_texts})
df_lemmatized.head()

Original Text  \
0  Sahil Shah identified as prime suspect in the ...   
1  Somewhere  between neurons  amp  narratives   ...   
2  Ppl trending National Embarrassment CBI   U R ...   
3  We all are missing your need  \n SushantSinghR...   
4  Hi All \n\nPlease sign this petition \n\n Just...   

                                     Lemmatized Text  
0  sahil shah identified a prime suspect in the d...  
1  somewhere between neuron amp narrative sushant...  
2  ppl trending national embarrassment cbi u r th...  
3  we all are missing your need sushantsinghrajpu...  
4  hi all please sign this petition justiceforsus...

In [90]:
lemmatized_full_text = " ".join(lemmatized_texts)
print("Lemmatized Text (First 500 chars):", lemmatized_full_text[:500])

Lemmatized Text (First 500 chars): sahil shah identified a prime suspect in the drug probe of sushant singh rajput case sushantsinghrajput sahilshah bollywoodbubble somewhere between neuron amp narrative sushantsinghrajput sushanting thebesthero warrior ssr ppl trending national embarrassment cbi u r the embarrassment plz use ur brain no investigating agency in the give update the public only the petitioner u want update ask the family of sushantsinghrajput sushantjustice selfmusing tkhere truth blve manifest we all are missing y


##### Observation:
The notebook applies lemmatization to text data to normalize words to their root forms. This reduces the dimensionality of the text, ensuring that variations of the same word (e.g., "running" and "run") are treated as a single term. The lemmatization process is particularly useful in sentiment analysis and other NLP tasks as it helps improve model accuracy by reducing redundancy in textual data.

##### Conclusion:
Lemmatization enhances text preprocessing by standardizing words to their base forms, leading to better feature extraction and model performance. It helps in reducing vocabulary size, improving generalization, and ultimately making sentiment classification more accurate and efficient.

### Cleaned_Text Process

In [199]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower() 
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  
    tokens = text.split()  
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return " ".join(tokens)




In [200]:
data["cleaned_text"] = data["text"].astype(str).apply(clean_text)
data[["text", "cleaned_text"]].head()

text  \
0  Sahil Shah identified as prime suspect in the ...   
1  Somewhere' between neurons &amp; narratives......   
2  Ppl trending National Embarrassment CBI - U R ...   
3  We all are missing your need. \n#SushantSinghR...   
4  Hi All,\n\nPlease sign this petition.\n\n#Just...   

                                        cleaned_text  
0  sahil shah identified prime suspect drug probe...  
1  somewhere neuron amp narrative sushantsinghraj...  
2  ppl trending national embarrassment cbi u r em...  
3  missing need sushantsinghrajput justiceforsush...  
4  hi please sign petition justiceforsushantsingh...

##### **Observation:**  
This function processes raw text data by converting it to lowercase, removing URLs, and eliminating non-alphabetic characters. Additionally, it tokenizes the text, removes stopwords, and applies lemmatization to standardize words to their base forms. This ensures that only meaningful words remain, reducing noise in the dataset and improving text analysis.  

##### **Conclusion:**  
By applying stopword removal and lemmatization, the function enhances the quality of text data, making it more suitable for NLP tasks such as sentiment analysis. The cleaned text is more structured and consistent, improving the performance of machine learning models by reducing redundancy and ensuring better generalization.

In [201]:
print(data.columns) 


Index(['text', 'processed_tweet', 'cleaned_text', 'neg', 'neu', 'pos',
       'compound', 'sentiment'],
      dtype='object')


In [203]:
data['cleaned_text'] = np.vectorize(clean_text)(data['text'])
data.head()


text  \
0  Sahil Shah identified as prime suspect in the ...   
1  Somewhere' between neurons &amp; narratives......   
2  Ppl trending National Embarrassment CBI - U R ...   
3  We all are missing your need. \n#SushantSinghR...   
4  Hi All,\n\nPlease sign this petition.\n\n#Just...   

                                     processed_tweet  \
0  sahil shah identified as prime suspect in the ...   
1  somewhere between neurons amp narratives susha...   
2  ppl trending national embarrassment cbi the em...   
3  we all are missing your need sushantsinghrajpu...   
4  hi all please sign this petition justiceforsus...   

                                        cleaned_text    neg    neu    pos  \
0  sahil shah identified prime suspect drug probe...  0.145  0.855  0.000   
1  somewhere neuron amp narrative sushantsinghraj...  0.000  1.000  0.000   
2  ppl trending national embarrassment cbi u r em...  0.179  0.741  0.080   
3  missing need sushantsinghrajput justiceforsush...  0.128  0.872  0.000   
4  hi please sign petition justiceforsushantsingh...  0.139  0.723  0.139   

   compound sentiment  
0   -0.2960  negative  
1    0.0000   neutral  
2   -0.6369  negative  
3   -0.2960  negative  
4    0.0000   neutral

### Wordnet 

In [94]:
import pandas as pd
from nltk.corpus import wordnet
original_texts = []
wordnet_transformed = []

for text in data["cleaned_text"].dropna():  
    words = text.split()
    if not words:
        continue

    synonyms = []
    for word in words:
        syns = wordnet.synsets(word)
        if syns:
            synonyms.append(syns[0].lemmas()[0].name())  
        else:
            synonyms.append(word)  

    original_texts.append(text)
    wordnet_transformed.append(" ".join(synonyms))



In [95]:
df_wordnet = pd.DataFrame({"Original Text": original_texts, "WordNet Synonyms": wordnet_transformed})
df_wordnet.head(10)

Original Text  \
0  sahil shah identified prime suspect drug probe...   
1  somewhere neuron amp narrative sushantsinghraj...   
2  ppl trending national embarrassment cbi u r em...   
3  missing need sushantsinghrajput justiceforsush...   
4  hi please sign petition justiceforsushantsingh...   
5  thank unconditional love amp support unbearabl...   
6  nationalembarrassmentcbi pmnarendramodi sushan...   
7  hi please sign petition justiceforsushantsingh...   
8  remember went crazy joy first shared itsssr lo...   
9  smitaparikh itsssr family lawyer must take ser...   

                                    WordNet Synonyms  
0  sahil Shah identify prime suspect drug probe s...  
1  somewhere nerve_cell ampere narrative sushants...  
2  ppl swerve national embarrassment cbi uracil r...  
3  miss need sushantsinghrajput justiceforsushant...  
4  hello please sign request justiceforsushantsin...  
5  thank unconditional love ampere support intole...  
6  nationalembarrassmentcbi pmnarendramodi sushan...  
7  hello please sign request justiceforsushantsin...  
8  remember travel crazy joy first share itsssr l...  
9  smitaparikh itsssr family lawyer must return s...

##### **Observation:**  
The dataset contains cleaned tweet text, but words may still have multiple variations and meanings. This function uses WordNet to replace words with their most common synonym, ensuring a more standardized representation of textual data. By mapping words to their primary synonyms, it helps reduce vocabulary size and aligns similar words, making text analysis more consistent.  

##### **Conclusion:**  
By transforming words using WordNet synonyms, this function enhances textual data quality, leading to better feature representation in NLP tasks. This process helps improve sentiment analysis and text classification models by reducing variations in word forms and ensuring more meaningful word substitutions. However, relying on the first synonym may sometimes alter the original meaning, so careful validation is necessary.

In [96]:
tokenized_tweet = data['cleaned_text'].apply(lambda x: x.split())
tokenized_tweet.head()


0    [sahil, shah, identified, prime, suspect, drug...
1    [somewhere, neuron, amp, narrative, sushantsin...
2    [ppl, trending, national, embarrassment, cbi, ...
3    [missing, need, sushantsinghrajput, justicefor...
4    [hi, please, sign, petition, justiceforsushant...
Name: cleaned_text, dtype: object

In [97]:
tokenized_tweet = data['processed_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()


0    [sahil, shah, identified, as, prime, suspect, ...
1    [somewhere, between, neurons, amp, narratives,...
2    [ppl, trending, national, embarrassment, cbi, ...
3    [we, all, are, missing, your, need, sushantsin...
4    [hi, all, please, sign, this, petition, justic...
Name: processed_tweet, dtype: object

In [98]:
for i in range(len(tokenized_tweet)):
    

     tokenized_tweet[i] = ' '.join(tokenized_tweet[i]) 
    
    
     #tokenized_tweet[i] = ' '.join([word for word in tokenized_tweet[i] if word not in stopwords])  
    

data['processed_tweet'] = tokenized_tweet
data.head()


Unnamed: 0         username  \
0           0      bollybubble   
1           1    sushantssmile   
2           2   Namita67793491   
3           3  being_grapher04   
4           4    sushantssmile   

                                         description             location  \
0  Bollywood Bubble - Your One Stop Destination F...  Mumbai, Maharashtra   
1  ᴡᴀɴᴛɪɴɢ ᴛᴏ ʟɪᴠᴇ ᴀs ʏᴏᴜʀ sᴍɪʟᴇ\nᴊᴜsᴛ ᴏɴᴇ ʟᴀsᴛ ᴛ...                    ∞   
2                                    Justice for SSR                  NaN   
3  Photographer.\nSushant Singh Rajput's Fan\nSSR...                 भारत   
4  ᴡᴀɴᴛɪɴɢ ᴛᴏ ʟɪᴠᴇ ᴀs ʏᴏᴜʀ sᴍɪʟᴇ\nᴊᴜsᴛ ᴏɴᴇ ʟᴀsᴛ ᴛ...                    ∞   

   following  followers  totaltweets  retweetcount  \
0        440     132406        51202             1   
1        369        412        11591            17   
2         66         89        20550             5   
3         82         27           20             0   
4        369        412        11591             1   

                                                text  \
0  Sahil Shah identified as prime suspect in the ...   
1  Somewhere' between neurons &amp; narratives......   
2  Ppl trending National Embarrassment CBI - U R ...   
3  We all are missing your need. \n#SushantSinghR...   
4  Hi All,\n\nPlease sign this petition.\n\n#Just...   

                                            hashtags  \
0  ['SushantSinghRajput', 'SahilShah', 'Bollywood...   
1  ['SushantSinghRajput', 'sushanting', 'thebesth...   
2                                                 []   
3  ['SushantSinghRajput', 'JusticeForSushantSingh...   
4  ['JusticeForSushantSinghRajput', 'SushantSingh...   

                                     processed_tweet  \
0  sahil shah identified as prime suspect in the ...   
1  somewhere between neurons amp narratives susha...   
2  ppl trending national embarrassment cbi the em...   
3  we all are missing your need sushantsinghrajpu...   
4  hi all please sign this petition justiceforsus...   

                                        cleaned_text  
0  sahil shah identified prime suspect drug probe...  
1  somewhere neuron amp narrative sushantsinghraj...  
2  ppl trending national embarrassment cbi u r em...  
3  missing need sushantsinghrajput justiceforsush...  
4  hi please sign petition justiceforsushantsingh...

In [99]:
data=data[['text','processed_tweet','cleaned_text']]
data


text  \
0     Sahil Shah identified as prime suspect in the ...   
1     Somewhere' between neurons &amp; narratives......   
2     Ppl trending National Embarrassment CBI - U R ...   
3     We all are missing your need. \n#SushantSinghR...   
4     Hi All,\n\nPlease sign this petition.\n\n#Just...   
...                                                 ...   
2995  @ShaheenaMd1 @Srishti15802104 Rhea Chakarborty...   
2996  Its #NewMoon on 12th Apr. New Moon brings in n...   
2997  🔱 Trend Activity Today 🔱\n\n Rhea Planted In S...   
2998  @REHANASUSHANT @ips_nupurprasad @AmitShah @DoP...   
2999  @its_your_Komal @SSR_IS_HERE @ips_nupurprasad ...   

                                        processed_tweet  \
0     sahil shah identified as prime suspect in the ...   
1     somewhere between neurons amp narratives susha...   
2     ppl trending national embarrassment cbi the em...   
3     we all are missing your need sushantsinghrajpu...   
4     hi all please sign this petition justiceforsus...   
...                                                 ...   
2995  rhea chakarborty is an extortionist she looted...   
2996  its newmoon on th apr new moon brings in new e...   
2997  trend activity today rhea planted in ssr life ...   
2998  rhea chakarborty is an extortionist she looted...   
2999  rhea chakarborty is an extortionist she looted...   

                                           cleaned_text  
0     sahil shah identified prime suspect drug probe...  
1     somewhere neuron amp narrative sushantsinghraj...  
2     ppl trending national embarrassment cbi u r em...  
3     missing need sushantsinghrajput justiceforsush...  
4     hi please sign petition justiceforsushantsingh...  
...                                                 ...  
2995  shaheenamd srishti rhea chakarborty extortioni...  
2996  newmoon th apr new moon brings new energy new ...  
2997  trend activity today rhea planted ssr life sus...  
2998  rehanasushant ipsnupurprasad amitshah doptgoi ...  
2999  itsyourkomal ssrishere ipsnupurprasad rhea cha...  

[3000 rows x 3 columns]

### Lexicon-Based Sentiment Analysis Using VADER

In [120]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Apply sentiment analysis
data['neg'] = data['cleaned_text'].apply(lambda x: analyzer.polarity_scores(str(x))['neg'])
data['neu'] = data['cleaned_text'].apply(lambda x: analyzer.polarity_scores(str(x))['neu'])
data['pos'] = data['cleaned_text'].apply(lambda x: analyzer.polarity_scores(str(x))['pos'])
data['compound'] = data['cleaned_text'].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])

# Display results
data[['cleaned_text', 'neg', 'pos', 'compound']].head()


cleaned_text    neg    pos  compound
0  sahil shah identified prime suspect drug probe...  0.145  0.000   -0.2960
1  somewhere neuron amp narrative sushantsinghraj...  0.000  0.000    0.0000
2  ppl trending national embarrassment cbi u r em...  0.179  0.080   -0.6369
3  missing need sushantsinghrajput justiceforsush...  0.128  0.000   -0.2960
4  hi please sign petition justiceforsushantsingh...  0.139  0.139    0.0000

In [101]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)   

data['neg'] = data['processed_tweet'].apply(lambda x:analyzer.polarity_scores(x)['neg'])
data['neu'] = data['processed_tweet'].apply(lambda x:analyzer.polarity_scores(x)['neu'])
data['pos'] = data['processed_tweet'].apply(lambda x:analyzer.polarity_scores(x)['pos'])
data['compound'] = data['processed_tweet'].apply(lambda x:analyzer.polarity_scores(x)['compound'])
data[['processed_tweet','neg','pos','compound']].head()


processed_tweet    neg    pos  compound
0  sahil shah identified as prime suspect in the ...  0.115  0.000   -0.2960
1  somewhere between neurons amp narratives susha...  0.000  0.000    0.0000
2  ppl trending national embarrassment cbi the em...  0.225  0.073   -0.7506
3  we all are missing your need sushantsinghrajpu...  0.180  0.000   -0.2960
4  hi all please sign this petition justiceforsus...  0.131  0.131    0.0000

##### **Observation:**  
The dataset contains preprocessed tweet text, and sentiment analysis is applied using the VADER (Valence Aware Dictionary and sEntiment Reasoner) model. VADER assigns four sentiment scores to each text: negative (`neg`), neutral (`neu`), positive (`pos`), and compound (`compound`). The `compound` score represents the overall sentiment by combining the individual scores. This method is effective for analyzing short texts like tweets, which often contain informal language, emojis, and slang.  

##### **Conclusion:**  
The VADER-based sentiment analysis provides a quantitative measure of sentiment for each tweet. This allows for categorizing tweets as positive, negative, or neutral based on the `compound` score. Since VADER is optimized for social media text, it handles negations, capitalization, and intensity well. This sentiment analysis helps in various NLP tasks, including opinion mining and trend analysis, improving the understanding of user sentiments in textual data.

### Sentiment Classification Using VADER

In [102]:
def getAnalysis(score):
    if score < 0:
      return 'negative'
    elif score == 0:
      return 'neutral'
    else:
      return 'positive'

data['sentiment'] = data['compound'].apply(getAnalysis)
# Show the dataframe
data[['processed_tweet','neg','pos','compound','sentiment','cleaned_text']].head()


processed_tweet    neg    pos  compound  \
0  sahil shah identified as prime suspect in the ...  0.115  0.000   -0.2960   
1  somewhere between neurons amp narratives susha...  0.000  0.000    0.0000   
2  ppl trending national embarrassment cbi the em...  0.225  0.073   -0.7506   
3  we all are missing your need sushantsinghrajpu...  0.180  0.000   -0.2960   
4  hi all please sign this petition justiceforsus...  0.131  0.131    0.0000   

  sentiment                                       cleaned_text  
0  negative  sahil shah identified prime suspect drug probe...  
1   neutral  somewhere neuron amp narrative sushantsinghraj...  
2  negative  ppl trending national embarrassment cbi u r em...  
3  negative  missing need sushantsinghrajput justiceforsush...  
4   neutral  hi please sign petition justiceforsushantsingh...

##### **Observation:**  
The dataset now includes a categorical sentiment label derived from the `compound` score calculated by the VADER sentiment analysis model. The function `getAnalysis()` classifies the sentiment into three categories:  
- **Negative** (if `compound` < 0)  
- **Neutral** (if `compound` = 0)  
- **Positive** (if `compound` > 0)  

This categorization helps in simplifying sentiment interpretation and making it easier to analyze patterns in the data.  

##### **Conclusion:**  
By assigning a sentiment label to each tweet, this function enhances sentiment classification, making it more interpretable for downstream analysis. This structured categorization is useful for visualization, trend detection, and training machine learning models for automated sentiment prediction. The approach effectively converts numerical sentiment scores into human-readable categories, improving the usability of the dataset.

###  Count Vectorizer

In [137]:
sentences = data['cleaned_text'].dropna().tolist()[:5]  

for i, sentence in enumerate(sentences):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([sentence])
    feature_names = vectorizer.get_feature_names_out() 
    df_sentence = pd.DataFrame(X.toarray(), columns=feature_names).T
    df_sentence.columns = [f"TF-IDF Score for Sentence {i+1}"]
    
    print(f"Sentence {i+1}: {sentence}")
    display(df_sentence)


Sentence 1: sahil shah identified prime suspect drug probe sushant singh rajput case sushantsinghrajput sahilshah bollywoodbubble


TF-IDF Score for Sentence 1
bollywoodbubble                               1
case                                          1
drug                                          1
identified                                    1
prime                                         1
probe                                         1
rajput                                        1
sahil                                         1
sahilshah                                     1
shah                                          1
singh                                         1
sushant                                       1
sushantsinghrajput                            1
suspect                                       1

Sentence 2: somewhere neuron amp narrative sushantsinghrajput sushanting thebesthero warriorsssr


TF-IDF Score for Sentence 2
amp                                           1
narrative                                     1
neuron                                        1
somewhere                                     1
sushanting                                    1
sushantsinghrajput                            1
thebesthero                                   1
warriorsssr                                   1

Sentence 3: ppl trending national embarrassment cbi u r embarrassment plz use ur brainsno investigating agency give update public petitioner u want update ask family sushantsinghrajput sushantjustice selfmusing tkheretruth blvemanifest


TF-IDF Score for Sentence 3
agency                                        1
ask                                           1
blvemanifest                                  1
brainsno                                      1
cbi                                           1
embarrassment                                 2
family                                        1
give                                          1
investigating                                 1
national                                      1
petitioner                                    1
plz                                           1
ppl                                           1
public                                        1
selfmusing                                    1
sushantjustice                                1
sushantsinghrajput                            1
tkheretruth                                   1
trending                                      1
update                                        2
ur                                            1
use                                           1
want                                          1

Sentence 4: missing need sushantsinghrajput justiceforsushantsinghrajput ssrwarriors sushantkojusticekabmilega sushant itsssr ssrwarriors gauritweets justiceforsush amitmuny thelegendssr fightssrianssr fightforssr jagdeepsingh


TF-IDF Score for Sentence 4
amitmuny                                                1
fightforssr                                             1
fightssrianssr                                          1
gauritweets                                             1
itsssr                                                  1
jagdeepsingh                                            1
justiceforsush                                          1
justiceforsushantsinghrajput                            1
missing                                                 1
need                                                    1
ssrwarriors                                             2
sushant                                                 1
sushantkojusticekabmilega                               1
sushantsinghrajput                                      1
thelegendssr                                            1

Sentence 5: hi please sign petition justiceforsushantsinghrajput sushantsinghrajput censorboard boycott movie nyaay sign petition via changeorgindia


TF-IDF Score for Sentence 5
boycott                                                 1
censorboard                                             1
changeorgindia                                          1
hi                                                      1
justiceforsushantsinghrajput                            1
movie                                                   1
nyaay                                                   1
petition                                                2
please                                                  1
sign                                                    2
sushantsinghrajput                                      1
via                                                     1

##### **Observation:**  
The code applies **Term Frequency-Inverse Document Frequency (TF-IDF)** vectorization on the first five sentences from the cleaned dataset. This method converts text into numerical values based on word importance. The `CountVectorizer` tokenizes the text and creates a vocabulary, then `fit_transform()` encodes the words into a sparse matrix representation. The transformed data is structured in a dataframe, where each row represents a word and its corresponding frequency-based score for each sentence.  

##### **Conclusion:**  
TF-IDF helps in identifying the most relevant words in each sentence by reducing the impact of commonly used words while emphasizing rare but significant terms. This transformation is useful for NLP tasks like text classification, clustering, and sentiment analysis. By converting textual data into a structured numerical format, it improves the performance of machine learning models, making text processing more efficient.

 ### TF-IDF Vectorizer

In [122]:

sentences = data['cleaned_text'].dropna().tolist()[:5]  # Taking first 5 sentences for demonstration

for i, sentence in enumerate(sentences):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([sentence])
    feature_names = vectorizer.get_feature_names_out()
    df_sentence = pd.DataFrame(X.toarray(), columns=feature_names).T
    df_sentence.columns = [f"TF-IDF Score for Sentence {i+1}"]
    
    print(f"\nSentence {i+1}: {sentence}\n")
    display(df_sentence.sort_values(by=df_sentence.columns[0], ascending=False))



Sentence 1: sahil shah identified prime suspect drug probe sushant singh rajput case sushantsinghrajput sahilshah bollywoodbubble



TF-IDF Score for Sentence 1
bollywoodbubble                        0.267261
case                                   0.267261
drug                                   0.267261
identified                             0.267261
prime                                  0.267261
probe                                  0.267261
rajput                                 0.267261
sahil                                  0.267261
sahilshah                              0.267261
shah                                   0.267261
singh                                  0.267261
sushant                                0.267261
sushantsinghrajput                     0.267261
suspect                                0.267261


Sentence 2: somewhere neuron amp narrative sushantsinghrajput sushanting thebesthero warriorsssr



TF-IDF Score for Sentence 2
amp                                    0.353553
narrative                              0.353553
neuron                                 0.353553
somewhere                              0.353553
sushanting                             0.353553
sushantsinghrajput                     0.353553
thebesthero                            0.353553
warriorsssr                            0.353553


Sentence 3: ppl trending national embarrassment cbi u r embarrassment plz use ur brainsno investigating agency give update public petitioner u want update ask family sushantsinghrajput sushantjustice selfmusing tkheretruth blvemanifest



TF-IDF Score for Sentence 3
update                                 0.371391
embarrassment                          0.371391
agency                                 0.185695
ppl                                    0.185695
use                                    0.185695
ur                                     0.185695
trending                               0.185695
tkheretruth                            0.185695
sushantsinghrajput                     0.185695
sushantjustice                         0.185695
selfmusing                             0.185695
public                                 0.185695
plz                                    0.185695
ask                                    0.185695
petitioner                             0.185695
national                               0.185695
investigating                          0.185695
give                                   0.185695
family                                 0.185695
cbi                                    0.185695
brainsno                               0.185695
blvemanifest                           0.185695
want                                   0.185695


Sentence 4: missing need sushantsinghrajput justiceforsushantsinghrajput ssrwarriors sushantkojusticekabmilega sushant itsssr ssrwarriors gauritweets justiceforsush amitmuny thelegendssr fightssrianssr fightforssr jagdeepsingh



TF-IDF Score for Sentence 4
ssrwarriors                                      0.471405
amitmuny                                         0.235702
fightforssr                                      0.235702
fightssrianssr                                   0.235702
gauritweets                                      0.235702
itsssr                                           0.235702
jagdeepsingh                                     0.235702
justiceforsush                                   0.235702
justiceforsushantsinghrajput                     0.235702
missing                                          0.235702
need                                             0.235702
sushant                                          0.235702
sushantkojusticekabmilega                        0.235702
sushantsinghrajput                               0.235702
thelegendssr                                     0.235702


Sentence 5: hi please sign petition justiceforsushantsinghrajput sushantsinghrajput censorboard boycott movie nyaay sign petition via changeorgindia



TF-IDF Score for Sentence 5
petition                                         0.471405
sign                                             0.471405
boycott                                          0.235702
censorboard                                      0.235702
changeorgindia                                   0.235702
hi                                               0.235702
justiceforsushantsinghrajput                     0.235702
movie                                            0.235702
nyaay                                            0.235702
please                                           0.235702
sushantsinghrajput                               0.235702
via                                              0.235702

##### **Observation:**  
The code uses the **TF-IDF (Term Frequency-Inverse Document Frequency) Vectorizer** to convert the first five cleaned sentences into numerical representations. Unlike simple word frequency counts, TF-IDF assigns importance to words based on their occurrence in a document relative to their presence across all documents. The transformed data is structured in a dataframe, with each row representing a word and its corresponding **TF-IDF score** for a given sentence. The words are then sorted in descending order based on their importance in the sentence.  

##### **Conclusion:**  
TF-IDF is an effective method for feature extraction in NLP, helping identify key terms in text data while down-weighting commonly occurring words. This technique is widely used in text classification, sentiment analysis, and search engine ranking algorithms. By converting text into meaningful numerical features, it enhances the ability of machine learning models to extract insights and improve prediction accuracy.

### Word2Vec CBOW Model

In [157]:
tokenized_sentences = [sentence.split() for sentence in data['cleaned_text'].dropna().tolist()]
word2vec_cbow = Word2Vec(sentences=tokenized_sentences, vector_size=5, window=5, min_count=2, workers=4, sg=0)
word2vec_cbow.save("word2vec_cbow.model")

vocabulary_size = len(word2vec_cbow.wv)
vocabulary_words = list(word2vec_cbow.wv.index_to_key)[:10]

In [158]:
print(f"Vocabulary Size: {vocabulary_size}"),
print(f"Sample Vocabulary Words: {vocabulary_words}")

Vocabulary Size: 2019
Sample Vocabulary Words: ['sushantsinghrajput', 'ssr', 'justice', 'case', 'sushant', 'u', 'month', 'ipsnupurprasad', 'itsssr', 'please']


In [159]:

word_to_check = "sushantsinghrajput"  # Choose a word from vocabulary
if word_to_check in word2vec_cbow.wv:
     similar_words = word2vec_cbow.wv.most_similar(word_to_check, topn=5)
     similar_words
else:
     word_to_check
similar_words



[('murder', 0.9856992959976196),
 ('humble', 0.9794324636459351),
 ('light', 0.9788599610328674),
 ('khatri', 0.9784164428710938),
 ('case', 0.9679438471794128)]

In [166]:
word_vectors = {word: word2vec_cbow.wv[word] for word in word2vec_cbow.wv.index_to_key}
df_word2vec = pd.DataFrame.from_dict(word_vectors, orient='index')
df_word2vec

0         1         2         3         4
sushantsinghrajput  -0.827851  0.725125  1.435131 -2.144818  0.478823
ssr                 -2.313330  1.371810  3.388250 -1.612666  1.982890
justice             -2.818081  3.281250  4.483382 -1.873078  2.504335
case                -1.656664  0.304237  2.300199 -2.618665  0.533119
sushant              3.075700  0.671475  2.620779  0.146619  3.310945
...                       ...       ...       ...       ...       ...
probably             0.206997  0.111920  0.194363 -0.080298  0.163165
list                 0.049833 -0.070205  0.240174 -0.137547  0.157701
mnths                0.005318  0.213868  0.190653 -0.077909  0.045344
gaye                 0.219882  0.435167  0.495592 -0.294301  0.263656
maharashtralockdown  0.076574 -0.086994  0.056364 -0.035784 -0.147406

[2019 rows x 5 columns]

In [161]:
sentences = data['cleaned_text'].dropna().tolist()[:2] 

for i, sentence in enumerate(sentences):
    words_in_sentence = set(sentence.lower().replace(".", "").split())
    relevant_words = [word for word in words_in_sentence if word in word_vectors]
    if not relevant_words:
        print(f"Sentence {i+1}: \"{sentence}\"")
        print("⚠ No words from this sentence found in the Word2Vec vocabulary.\n")
        continue
    word_embeddings = {word: word_vectors[word] for word in relevant_words}
    df_embeddings = pd.DataFrame(word_embeddings).T

    print(f"Sentence {i+1}: \"{sentence}\"")
    display(df_embeddings)


Sentence 1: "sahil shah identified prime suspect drug probe sushant singh rajput case sushantsinghrajput sahilshah bollywoodbubble"


0         1         2         3         4
prime               0.555009  0.759490  0.947453 -0.769874  0.572396
sushant             3.075700  0.671475  2.620779  0.146619  3.310945
identified          0.370238  0.663782  0.947511 -0.781368  0.330510
sahilshah           0.270520  0.338012  0.642159 -0.405600 -0.027033
suspect             0.492208  0.656498  0.988394 -0.706450  0.528302
sahil               0.561181  0.853309  0.936359 -0.830024  0.448207
drug                1.470642  1.854833  2.456648 -2.248820  1.032033
shah                0.555915  0.506350  0.851929 -0.961596  0.354382
probe               0.011600  0.467378  0.250616 -0.383417  0.113909
sushantsinghrajput -0.827851  0.725125  1.435131 -2.144818  0.478823
singh               1.092576  1.371535  2.735255  0.644173  4.215367
case               -1.656664  0.304237  2.300199 -2.618665  0.533119
rajput              0.499073  0.036431  3.378648 -0.000667  3.424372

Sentence 2: "somewhere neuron amp narrative sushantsinghrajput sushanting thebesthero warriorsssr"


0         1         2         3         4
amp                -0.293574  2.270041  1.987360 -1.762769  2.002347
warriorsssr         0.289103  0.633465  0.518608 -0.365509  0.329212
sushantsinghrajput -0.827851  0.725125  1.435131 -2.144818  0.478823
narrative           0.084542  0.270626 -0.170263 -0.143124  0.121451

##### **Observation:**  
The code trains a **Word2Vec Continuous Bag of Words (CBOW) model** using tokenized text data. CBOW learns word representations by predicting a target word based on surrounding context words. The model is trained with a **vector size of 5**, a **window of 5 words**, and **a minimum word frequency of 2**. After training, the vocabulary size is determined, and the 10 most frequent words are extracted. The code also checks for words similar to "sushantsinghrajput" using cosine similarity, retrieving the top 5 closest words in the learned embedding space. Finally, word vectors are stored in a dataframe for further analysis.  

### **Conclusion:**  
Word2Vec's CBOW model effectively captures word relationships and semantic meanings by learning word embeddings. This technique is useful for NLP tasks such as sentiment analysis, topic modeling, and recommendation systems. The generated word embeddings can be used to analyze word similarities, identify trends in text data, and improve text-based machine learning models by providing meaningful word representations.

### fasttext_cbow

In [162]:
sentences = data['cleaned_text'].dropna().tolist()[:5]
tokenized_sentences = [sentence.lower().split() for sentence in sentences]
fasttext_cbow = FastText(sentences=tokenized_sentences, vector_size=5, window=5, min_count=1, sg=0)

for i, sentence in enumerate(sentences):
    words_in_sentence = set(sentence.lower().split())
    relevant_words = [word for word in words_in_sentence if word in fasttext_cbow.wv]
    if not relevant_words:
        print(f"Sentence {i+1}: \"{sentence}\"")
        print("⚠ No words from this sentence found in the FastText vocabulary.\n")
        continue
    word_embeddings = {word: fasttext_cbow.wv[word] for word in relevant_words}
    df_embeddings = pd.DataFrame(word_embeddings).T  # Convert to DataFrame

    print(f"Sentence {i+1}: \"{sentence}\"")
    display(df_embeddings)


Sentence 1: "sahil shah identified prime suspect drug probe sushant singh rajput case sushantsinghrajput sahilshah bollywoodbubble"


0         1         2         3         4
prime              -0.000772  0.007245  0.015066  0.035980  0.030355
sushant             0.015615  0.006184  0.031984  0.013312 -0.012203
identified          0.000788  0.034562  0.008695 -0.013131  0.010890
sahilshah           0.045247 -0.031911  0.025787 -0.012387 -0.044926
suspect             0.033398  0.044553  0.009877 -0.024058  0.006145
sahil               0.015018 -0.035219  0.001761 -0.039008 -0.065141
drug                0.032604 -0.016370 -0.026321 -0.002816  0.013901
shah                0.015363 -0.033798 -0.025203  0.011938 -0.059118
probe               0.094335  0.011116  0.012017 -0.028521 -0.035427
sushantsinghrajput  0.007968 -0.001411 -0.000253  0.006777  0.017806
bollywoodbubble     0.009470  0.009728 -0.014272  0.013843  0.015241
singh              -0.012522 -0.002631  0.058205  0.019187 -0.051717
case                0.040481 -0.022235 -0.011780 -0.070373  0.008999
rajput              0.000842  0.006693  0.004969  0.016551  0.044268

Sentence 2: "somewhere neuron amp narrative sushantsinghrajput sushanting thebesthero warriorsssr"


0         1         2         3         4
somewhere          -0.007863 -0.000334  0.011531  0.023267  0.008544
neuron             -0.039211 -0.024083  0.020493 -0.008699 -0.010152
amp                 0.050987  0.034579 -0.001841 -0.032292  0.028891
warriorsssr         0.003994  0.026286  0.009763 -0.000994 -0.017513
sushanting          0.016314  0.027948  0.026215  0.003322 -0.002035
sushantsinghrajput  0.007968 -0.001411 -0.000253  0.006777  0.017806
narrative           0.006042  0.024802 -0.011227  0.000568  0.016698
thebesthero        -0.009693 -0.010778 -0.016520 -0.020343 -0.000161

Sentence 3: "ppl trending national embarrassment cbi u r embarrassment plz use ur brainsno investigating agency give update public petitioner u want update ask family sushantsinghrajput sushantjustice selfmusing tkheretruth blvemanifest"


0         1         2         3         4
petitioner          0.009837  0.049065 -0.010362  0.014797  0.032124
use                 0.025100 -0.054997  0.000297  0.002919  0.007782
give               -0.036974  0.007421  0.019946  0.049807  0.042164
public             -0.029934 -0.009475  0.008353  0.004797 -0.000839
national           -0.022601  0.006297  0.013527  0.008555  0.028954
ur                 -0.056730 -0.017150  0.035295  0.015025  0.051025
sushantjustice      0.020586  0.029034  0.017532  0.000920 -0.010464
plz                 0.020457 -0.006635 -0.001304 -0.032592 -0.014190
ask                -0.005921 -0.020897  0.010773  0.032793 -0.067728
trending           -0.005378 -0.003627 -0.018496 -0.009380 -0.005037
family             -0.040358 -0.033331  0.017221  0.010260 -0.023788
tkheretruth         0.004879  0.012009  0.001974  0.007524  0.000416
sushantsinghrajput  0.007968 -0.001411 -0.000253  0.006777  0.017806
want               -0.009503  0.042359 -0.009971 -0.030164  0.034857
u                  -0.065343  0.034334 -0.043288 -0.062252 -0.133055
embarrassment       0.023183  0.029101 -0.002842  0.006165  0.033778
blvemanifest       -0.017930 -0.015026  0.010858 -0.000832  0.013753
update              0.019909 -0.004219 -0.023249  0.052439  0.011511
investigating      -0.006849 -0.009047 -0.011742 -0.008167 -0.001170
selfmusing          0.006558  0.013385  0.019253 -0.004355 -0.003888
r                   0.048723  0.154991 -0.015927 -0.045774  0.126791
cbi                 0.004496  0.047914 -0.002217 -0.020619  0.055801
ppl                -0.033929 -0.008678  0.037303  0.044629  0.062011
brainsno            0.000288 -0.018740 -0.036915 -0.011646  0.012075
agency             -0.007625  0.043438 -0.011164  0.004111 -0.002437

Sentence 4: "missing need sushantsinghrajput justiceforsushantsinghrajput ssrwarriors sushantkojusticekabmilega sushant itsssr ssrwarriors gauritweets justiceforsush amitmuny thelegendssr fightssrianssr fightforssr jagdeepsingh"


0         1         2         3         4
missing                      -0.006275  0.045289 -0.004752 -0.005455 -0.005704
sushant                       0.015615  0.006184  0.031984  0.013312 -0.012203
itsssr                        0.001568 -0.008975 -0.013057 -0.020655 -0.054513
ssrwarriors                   0.015000  0.014809  0.008845 -0.009573 -0.011976
gauritweets                   0.002954  0.015766 -0.014031 -0.050127  0.019656
justiceforsush               -0.003068  0.008482  0.002414 -0.020620 -0.000929
fightforssr                  -0.021403  0.017279  0.011662  0.005969  0.002913
need                         -0.021376 -0.016644  0.009048  0.020796 -0.068698
justiceforsushantsinghrajput  0.003308  0.000161  0.001371 -0.004044  0.013286
amitmuny                      0.023659 -0.019614 -0.012279  0.005126 -0.014337
jagdeepsingh                 -0.011815  0.002557  0.015016  0.011221 -0.010016
thelegendssr                 -0.013577  0.003253 -0.024959  0.041504  0.034696
sushantsinghrajput            0.007968 -0.001411 -0.000253  0.006777  0.017806
fightssrianssr                0.000301  0.004249 -0.020755 -0.009436  0.007449
sushantkojusticekabmilega     0.011210  0.000832  0.003733 -0.004934  0.002969

Sentence 5: "hi please sign petition justiceforsushantsinghrajput sushantsinghrajput censorboard boycott movie nyaay sign petition via changeorgindia"


0         1         2         3         4
nyaay                        -0.011010  0.051355 -0.017033  0.005837 -0.024050
justiceforsushantsinghrajput  0.003308  0.000161  0.001371 -0.004044  0.013286
via                          -0.004101 -0.008899  0.029737 -0.011034  0.043011
hi                           -0.129247 -0.058814  0.054535 -0.101841  0.069383
movie                         0.027491  0.052202 -0.021740  0.062466  0.045917
petition                      0.020726  0.045694 -0.019763  0.008027  0.049800
censorboard                  -0.029069 -0.064190  0.001092  0.024131 -0.010133
sushantsinghrajput            0.007968 -0.001411 -0.000253  0.006777  0.017806
changeorgindia               -0.012380  0.007453 -0.010500  0.030768  0.011146
please                       -0.005598 -0.047957 -0.016891 -0.062697 -0.044186
sign                          0.006252 -0.012365  0.002651 -0.026740 -0.038748
boycott                       0.015768 -0.037639 -0.011019 -0.019910  0.024771

In [163]:
vocabulary_size = len(fasttext_cbow.wv)
vocabulary_words = list(fasttext_cbow.wv.index_to_key)[:10]
print(f"Vocabulary Size: {vocabulary_size}"),
print("Top 10 Words in Vocabulary:", vocabulary_words)


Vocabulary Size: 68
Top 10 Words in Vocabulary: ['sushantsinghrajput', 'sushant', 'petition', 'ssrwarriors', 'embarrassment', 'sign', 'justiceforsushantsinghrajput', 'u', 'update', 'cbi']


In [167]:
word_vectors = {word: fasttext_cbow.wv[word] for word in fasttext_cbow.wv.index_to_key}
df_fasttext = pd.DataFrame.from_dict(word_vectors, orient='index')
df_fasttext


0         1         2         3         4
sushantsinghrajput  0.007968 -0.001411 -0.000253  0.006777  0.017806
sushant             0.015615  0.006184  0.031984  0.013312 -0.012203
petition            0.020726  0.045694 -0.019763  0.008027  0.049800
ssrwarriors         0.015000  0.014809  0.008845 -0.009573 -0.011976
embarrassment       0.023183  0.029101 -0.002842  0.006165  0.033778
...                      ...       ...       ...       ...       ...
petitioner          0.009837  0.049065 -0.010362  0.014797  0.032124
public             -0.029934 -0.009475  0.008353  0.004797 -0.000839
give               -0.036974  0.007421  0.019946  0.049807  0.042164
via                -0.004101 -0.008899  0.029737 -0.011034  0.043011
sahil               0.015018 -0.035219  0.001761 -0.039008 -0.065141

[68 rows x 5 columns]

##### **Observation:**  
The code trains a **FastText CBOW (Continuous Bag of Words) model** on tokenized sentences from the dataset. FastText, an extension of Word2Vec, enhances word embeddings by considering subword information, making it more effective for handling rare words, misspellings, and morphologically rich languages. The model is trained with a **vector size of 5**, a **context window of 5 words**, and includes all words appearing at least once (`min_count=1`). After training, the code extracts word embeddings for words present in the vocabulary and displays them in a structured DataFrame. If a sentence contains no recognized words, a warning message is shown.  

##### **Conclusion:**  
FastText CBOW improves over traditional Word2Vec by representing words as character n-grams, allowing it to generate embeddings for unseen or misspelled words. This makes it highly effective for noisy text, such as social media data. The generated embeddings can be used for various NLP tasks, including text classification, named entity recognition, and recommendation systems. By leveraging subword information, FastText enhances the accuracy of word similarity tasks and improves model generalization.

In [138]:
slices = [(data['sentiment'] == 'negative').sum(), (data['sentiment'] == 'positive').sum(), (data['sentiment'] == 'neutral').sum()]
slices


[970, 1563, 467]

### Sentiment Distribution Pie Chart

In [125]:
sentiments = ['Negative Sentiment','Positive Sentiment', 'Neutral Sentiment'] 
slices = [(data['sentiment'] == 'negative').sum(), (data['sentiment'] == 'positive').sum(), (data['sentiment'] == 'neutral').sum()]
colors = ['r','b','y'] 
plt.pie(slices, labels = sentiments, colors=colors, startangle=70, shadow = True,
        explode = (0, 0.1,0.2), radius = 1, autopct = '%1.2f%%') 
plt.legend(loc = "lower right")
plt.show() 


##### **Observation:**  
The code generates a **pie chart** to visualize the distribution of sentiment categories (Negative, Positive, and Neutral) in the dataset. The number of tweets in each sentiment category is counted using `.sum()` and represented as slices in the pie chart. The slices are color-coded (**red for negative, blue for positive, and yellow for neutral**) and slightly exploded for better visibility. The **percentage of each sentiment** is displayed using `autopct='%1.2f%%'`, and a legend is placed at the lower right corner.  

##### **Conclusion:**  
This visualization provides a clear **proportion of sentiment distribution** in the dataset. It helps in understanding the overall mood of the tweets and identifying sentiment trends. If one sentiment dominates, it could indicate bias in the dataset. Such insights are useful for sentiment analysis applications in areas like customer feedback analysis, brand monitoring, and social media trend analysis.

In [126]:
data.head()


text  \
0  Sahil Shah identified as prime suspect in the ...   
1  Somewhere' between neurons &amp; narratives......   
2  Ppl trending National Embarrassment CBI - U R ...   
3  We all are missing your need. \n#SushantSinghR...   
4  Hi All,\n\nPlease sign this petition.\n\n#Just...   

                                     processed_tweet  \
0  sahil shah identified as prime suspect in the ...   
1  somewhere between neurons amp narratives susha...   
2  ppl trending national embarrassment cbi the em...   
3  we all are missing your need sushantsinghrajpu...   
4  hi all please sign this petition justiceforsus...   

                                        cleaned_text    neg    neu    pos  \
0  sahil shah identified prime suspect drug probe...  0.145  0.855  0.000   
1  somewhere neuron amp narrative sushantsinghraj...  0.000  1.000  0.000   
2  ppl trending national embarrassment cbi u r em...  0.179  0.741  0.080   
3  missing need sushantsinghrajput justiceforsush...  0.128  0.872  0.000   
4  hi please sign petition justiceforsushantsingh...  0.139  0.723  0.139   

   compound sentiment  
0   -0.2960  negative  
1    0.0000   neutral  
2   -0.6369  negative  
3   -0.2960  negative  
4    0.0000   neutral

### **Word Cloud Visualization for Cleaned_text**

In [107]:
all_words = ' '.join([text for text in data['cleaned_text']])
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words) 
plt.figure(figsize=(10, 7)) 
plt.imshow(wordcloud, interpolation="bilinear") 
plt.axis('off')
plt.show()


##### **Observation:**  
The code generates a **Word Cloud**, a visual representation of the most frequently occurring words in the dataset. It first combines all cleaned text from the dataset into a single string. The `WordCloud` function then processes this text, adjusting the font size based on word frequency. The **larger the word appears, the more frequently it occurs in the dataset**. The final word cloud is displayed using `matplotlib`, with the words arranged randomly for better visualization.  

##### **Conclusion:**  
A **word cloud** provides a quick and intuitive way to identify the most commonly used words in the dataset. It is particularly useful for **exploratory data analysis (EDA)** in NLP tasks, allowing insights into key topics, themes, or trends within the text data. If many irrelevant or stopwords dominate the cloud, additional preprocessing (e.g., stopword removal, lemmatization) may be needed to improve meaningful word representation.

### Word Cloud for Processed Tweets

In [108]:
all_words = ' '.join([text for text in data['processed_tweet']])
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words) 
plt.figure(figsize=(10, 7)) 
plt.imshow(wordcloud, interpolation="bilinear") 
plt.axis('off')
plt.show()


## **Word Cloud for Processed Tweets**  

### **Observation:**  
This code generates a **Word Cloud** using the processed tweets stored in the `processed_tweet` column. It first concatenates all processed tweets into a single string and then applies the `WordCloud` function to visualize the most frequently occurring words. The word sizes represent their **frequency in the dataset**, with larger words appearing more often. The visualization is displayed using `matplotlib`, with an **800x500 resolution and a maximum font size of 110** for better readability.  

### **Conclusion:**  
The **word cloud** helps identify key themes and commonly used words in the dataset after preprocessing. Since this version uses **processed tweets**, it is expected to be cleaner and more meaningful than the raw text version. This technique is beneficial in **sentiment analysis, topic modeling, and trend identification** by highlighting the most prominent words in social media discussions. If stopwords still appear frequently, further text preprocessing may be required.

In [109]:
n_class = 3
n_tweet = 500

if n_class == 2:
    df_pos = data.copy()[data.sentiment == 'positive'][:n_tweet]
    df_neg = data.copy()[data.sentiment == 'negative'][:n_tweet]
    df_neu = pd.DataFrame()
    df = pd.concat([df_pos, df_neg], ignore_index=True).reset_index(drop=True)
elif n_class == 3:
    df_pos = data.copy()[data.sentiment == 'positive'][:n_tweet]
    df_neg = data.copy()[data.sentiment == 'negative'][:n_tweet]
    df_neu = data.copy()[data.sentiment == 'neutral'][:n_tweet]
    df = pd.concat([df_pos, df_neg, df_neu], ignore_index=True).reset_index(drop=True)


### **Train-Test Split for Sentiment Analysis**

In [192]:
X_train, X_test, y_train, y_test = train_test_split(data['processed_tweet'], data['sentiment'], test_size=0.33)

df_train = pd.DataFrame()
df_test = pd.DataFrame()

df_train['processed_tweet'] = X_train
df_train['sentiment'] = y_train
df_train = df_train.reset_index(drop=True)

df_test['processed_tweet'] = X_test
df_test['sentiment'] = y_test
df_test = df_test.reset_index(drop=True)
df_test


processed_tweet sentiment
0    dr gupta told sir that he was sure that sushan...  positive
1                good night warrior sushantsinghrajput  positive
2    we love you mere ssr and miss you every day ev...  positive
3    wonder who and why is cooking up the schizophr...  positive
4    the world can change by example and not by opi...  negative
..                                                 ...       ...
985  sushantsinghrajput mahacmcbiforssr monthsofssr...  positive
986  if sushantsinghrajput family is so sure that h...  negative
987  sush if possible please come back aapne pehle ...  positive
988  have recently explored place in gurgaon called...  negative
989  dr gupta told sir that he was sure that sushan...  positive

[990 rows x 2 columns]

##### **Observation:**  
The dataset is **split into training and testing sets** using the `train_test_split` function, with **33% of the data allocated for testing** and the remaining **67% for training**. The `processed_tweet` column serves as the input (features), while the `sentiment` column acts as the target (labels). The resulting train and test dataframes (`df_train` and `df_test`) are then reset with new indexing for better structure.  

##### **Conclusion:**  
A proper train-test split is essential to ensure that the sentiment analysis model generalizes well to unseen data. **Using a 67-33 split provides a balanced approach**, allowing the model to learn from sufficient data while maintaining a sizable portion for evaluation. This process helps **prevent overfitting and ensures reliable model performance assessment** before deployment.

### **Naive Bayes Sentiment Classification**

In [193]:
class TweetNBClassifier(object):

    def __init__(self, df_train):
        self.df_train = df_train
        self.df_pos = df_train.copy()[df_train.sentiment == 'positive']
        self.df_neg = df_train.copy()[df_train.sentiment == 'negative']
        self.df_neu = df_train.copy()[df_train.sentiment == 'neutral']

    def fit(self):
        Pr_pos = df_pos.shape[0]/self.df_train.shape[0]
        Pr_neg = df_neg.shape[0]/self.df_train.shape[0]
        Pr_neu = df_neu.shape[0]/self.df_train.shape[0]
        self.Prior  = (Pr_pos, Pr_neg, Pr_neu)

        self.pos_words = ' '.join(self.df_pos['processed_tweet'].tolist()).split()
        self.neg_words = ' '.join(self.df_neg['processed_tweet'].tolist()).split()
        self.neu_words = ' '.join(self.df_neu['processed_tweet'].tolist()).split()

        all_words = ' '.join(self.df_train['processed_tweet'].tolist()).split()

        self.vocab = len(Counter(all_words))

        wc_pos = len(' '.join(self.df_pos['processed_tweet'].tolist()).split())
        wc_neg = len(' '.join(self.df_neg['processed_tweet'].tolist()).split())
        wc_neu = len(' '.join(self.df_neu['processed_tweet'].tolist()).split())
        self.word_count = (wc_pos, wc_neg, wc_neu)
        return self


    def predict(self, df_test):
        class_choice = ['positive', 'negative', 'neutral']

        classification = []
        for tweet in df_test['processed_tweet']:
            text = tweet.split()

            val_pos = np.array([])
            val_neg = np.array([])
            val_neu = np.array([])
            for word in text:
                tmp_pos = np.log((self.pos_words.count(word)+1)/(self.word_count[0]+self.vocab))
                tmp_neg = np.log((self.neg_words.count(word)+1)/(self.word_count[1]+self.vocab))
                tmp_neu = np.log((self.neu_words.count(word)+1)/(self.word_count[2]+self.vocab))
                val_pos = np.append(val_pos, tmp_pos)
                val_neg = np.append(val_neg, tmp_neg)
                val_neu = np.append(val_neu, tmp_neu)

            val_pos = np.log(self.Prior[0]) + np.sum(val_pos)
            val_neg = np.log(self.Prior[1]) + np.sum(val_neg)
            val_neu = np.log(self.Prior[2]) + np.sum(val_neu)

            probability = (val_pos, val_neg, val_neu)
            classification.append(class_choice[np.argmax(probability)])
        return classification


    def score(self, feature, target):

        compare = []
        for i in range(0,len(feature)):
            if feature[i] == target[i]:
                tmp ='correct'
                compare.append(tmp)
            else:
                tmp ='incorrect'
                compare.append(tmp)
        r = Counter(compare)
        accuracy = r['correct']/(r['correct']+r['incorrect'])
        return accuracy


In [194]:
tnb = TweetNBClassifier(df_train)
tnb = tnb.fit()
predict = tnb.predict(df_test)
score = tnb.score(predict,df_test.sentiment.tolist())
print("Accuracy of model is: ",score)


Accuracy of model is:  0.908080808080808


##### **Observation:**  
A custom **Naïve Bayes classifier (`TweetNBClassifier`)** is implemented to classify tweets into **positive, negative, or neutral sentiments**. It calculates prior probabilities, tokenizes words, applies **Laplace smoothing**, and uses **Bayes' Theorem** to predict sentiment. The accuracy of predictions is evaluated against test data.  

##### **Conclusion:**  
The model is **efficient and interpretable**, making it suitable for sentiment analysis. Despite its simplicity, **it performs well** but can be improved with **TF-IDF, better negation handling, or deep learning techniques**.

### **Logistic Regression Sentiment Classification**  

In [186]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['sentiment'], test_size=0.2, random_state=42)
vocab = set(word for text in X_train for word in text.split())  
word_index = {word: idx for idx, word in enumerate(vocab)}

def text_to_features(texts, word_index):
    features = np.zeros((len(texts), len(word_index)))
    for i, text in enumerate(texts):
        word_counts = Counter(text.split())  
        for word, count in word_counts.items():
            if word in word_index:
                features[i, word_index[word]] = count  
    return features
X_train_features = text_to_features(X_train, word_index)
X_test_features = text_to_features(X_test, word_index) 

# Train Logistic Regression
clf = LogisticRegression(max_iter=500, solver='liblinear')
clf.fit(X_train_features, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_features)
accuracy = (y_pred == y_test).mean()
print(f"🔹 Accuracy using Cleaned Text Only: {accuracy * 100:.2f}%")


🔹 Accuracy using Cleaned Text Only: 95.83%


##### **Observation:**  
A **Logistic Regression model** is trained on **cleaned tweet text** using a **custom word count-based feature representation**. The vocabulary is built from the training data, and each text is converted into a **word frequency feature vector**. The model is trained and evaluated on the test dataset.  

##### **Conclusion:**  
Logistic Regression is a **simple yet effective** approach for sentiment classification. The accuracy indicates **reasonable performance**, but improvements like **TF-IDF weighting, n-grams, or deep learning models** could further enhance results.